# Audio Classification: Exploring CRNN

**A CRNN (Convolutional Recurrent Neural Network) combines two powerful ideas:**
1. CNN layers extract spatial features from the spectrogram
2. RNN layers (GRU/LSTM) learn temporal evolution of the sound over time

In this notebook, we build a CRNN by applying these two stages one after another.

In [32]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical